# EHSD Lab Exercise 1

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from get_tl_model import get_tl_model

In [ ]:
def load_ltspice(path = "../ltspice/circuit_model.asc"):
    """
    Loads the LTSpice model and parses the component values.

    Returns:
    lines (list[str]): list of strings containing all lines from the LTSpice model
    param_lines (list[str]): list of strings containing all .param lines from the LTSpice model
    param_idx (list[int]): list of integers containing the line index of each .param line
    """

    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    param_idx = []
    param_lines = []
    for i, line in enumerate(lines):
        if '.param' in line:
            param_idx.append(i)
            param_lines.append(line)

    print("------ Current Parameters ------")
    for line in param_lines:
        values = line.split('.param')[1].strip()
        print(values)

    return lines, param_idx, param_lines

def edit_ltspice(lines, param_lines, param_idx, component_values,
                 path: str = "../ltspice/circuit_model.asc"):
    """
    Edits the LTSpice model with the new component values.
    
    Args:
    lines (list[str]): list of strings containing all lines from the LTSpice model
    param_lines (list[str]): list of strings containing all .param lines from the LTSpice model
    param_idx (list[int]): list of integers containing the line index of each .param line
    component_values (dict): dictionary containing the new component values
    path (str): path to the LTSpice model file
    """

    for i, line in enumerate(param_lines):
        mod_line = line.split('.param')[1].strip().split(' ')

        for key, value in component_values.items():
            for j, param in enumerate(mod_line):
                param = param.split('=')[0]
                if key == param:
                    mod_line[j] = key + '=' + value
        mod_line = ' '.join(mod_line)
        
        lines[param_idx[i]] = lines[param_idx[i]].split('.param')[0] + '.param ' + mod_line + '\n'

    param_lines = []
    for i, line in enumerate(lines):
        if '.param' in line:
            param_lines.append(line)

    print("\n------ New Parameters ------")
    for line in param_lines:
        values = line.split('.param')[1].strip()
        print(values)

    usr_input = input("Write? (y/n): ")
    if usr_input == 'y':
        with open("../ltspice/circuit_model.asc", 'w', encoding='utf-8') as f:
            f.writelines(lines)

In [ ]:
# Case 1, side-by-side microstrip
case = 1
tl_model_params = get_tl_model(case=case)
print(f"------ TL Model Parameters (Case {case}) ------")
for key, value in tl_model_params.items():
    if isinstance(value, np.ndarray):
        print(f"{key}:\n", np.round(value, 2))
    else:
        print(f"{key}:\t", "{:.2e}".format(value))

QFN_params = {
    'R_pkg': '0.097',
    'L_pkg': '1.415n',
    'C_pkg': '0.139p'
}

SOIC_params = {
    'R_pkg': '0.038',
    'L_pkg': '5.012n',
    'C_pkg': '0.707p'
}

def get_params(component_values, qfn=False):
    params = component_values
    params['R1A'] = QFN_params['R_pkg'] if qfn else SOIC_params['R_pkg']
    params['L1A'] = QFN_params['L_pkg'] if qfn else SOIC_params['L_pkg']
    params['C2A'] = QFN_params['C_pkg'] if qfn else SOIC_params['C_pkg']
    params['C3A'] = QFN_params['C_pkg'] if qfn else SOIC_params['C_pkg']
    params['R3A'] = QFN_params['R_pkg'] if qfn else SOIC_params['R_pkg']
    params['L2A'] = QFN_params['L_pkg'] if qfn else SOIC_params['L_pkg']
    return params

prop_speed = 1.599e8    # 159.9 m/s
length = 0.10           # 10 cm
TD = str(length / prop_speed)
print(TD)

component_values = {
    'C1A': '6.03p',
    'R1A': '0', # Pkg
    'L1A': '0', # Pkg
    'C2A': '0', # Pkg
    'R2A': '39',
    'Z0A': '50', # Assume 50 ohm
    'C3A': '0',
    'R3A': '0',
    'L2A': '0',
    'C4A': '4.21p'
}

component_values = get_params(component_values, qfn=True)

lines, param_idx, param_lines = load_ltspice()
edit_ltspice(lines, param_lines, param_idx, component_values)

In [ ]:
for case in range(1, 4):
    tl_model_params = get_tl_model(case=case)
    print(f"------ TL Model Parameters (Case {case}) ------")
    for key, value in tl_model_params.items():
        if isinstance(value, np.ndarray):
            print(f"{key}:\n", np.round(value, 2))
        else:
            print(f"{key}:\t", "{:.2e}".format(value))